**Topic**
- Machine Learning

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("MachineLearning").getOrCreate()
spark

In [4]:
training = spark.read.csv("test5.csv", header=True, inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|   Shayan| 31|        10| 30000|
|  Pranjit| 30|         8| 25000|
|Dwiprahar| 29|         4| 20000|
|   Samrat| 24|         3| 20000|
| Prasanta| 21|         1| 15000|
|   Tanmoy| 23|         2| 18000|
+---------+---+----------+------+



In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [9]:
from pyspark.ml.feature import VectorAssembler
featureAssembler=VectorAssembler(inputCols=['Age','Experience'],outputCol="Independent Features")
output=featureAssembler.transform(training)
output.show()

+---------+---+----------+------+--------------------+
|     Name|Age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|   Shayan| 31|        10| 30000|         [31.0,10.0]|
|  Pranjit| 30|         8| 25000|          [30.0,8.0]|
|Dwiprahar| 29|         4| 20000|          [29.0,4.0]|
|   Samrat| 24|         3| 20000|          [24.0,3.0]|
| Prasanta| 21|         1| 15000|          [21.0,1.0]|
|   Tanmoy| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [10]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [11]:
finalized=output.select("Independent Features","Salary")
finalized.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [27]:
from pyspark.ml.regression import LinearRegression as Lr
train_data,test_data=finalized.randomSplit([0.75,0.25])
regressor=Lr(featuresCol='Independent Features',labelCol="Salary")
regressor=regressor.fit(train_data)

In [28]:
regressor.coefficients

DenseVector([2666.6667, -2000.0])

In [29]:
regressor.intercept

-38833.33333337073

In [31]:
predict=regressor.evaluate(test_data)
predict.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [29.0,4.0]| 20000| 30500.00000000719|
|         [31.0,10.0]| 30000|23833.333333330404|
+--------------------+------+------------------+



In [32]:
predict.meanAbsoluteError,predict.meanSquaredError

(8333.333333338393, 74138888.88898243)